In [2]:
print('hi')

hi


In [37]:
pip install xgboost xgboost[spark]


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# 1. Start Spark session
spark = SparkSession.builder.appName("XGBoostSparkExample").getOrCreate()

# 2. Create a small sample dataset
data = [
    (1.0, 2.0, 0),
    (2.0, 1.0, 1),
    (1.5, 1.5, 0),
    (3.0, 3.0, 1),
    (2.5, 0.5, 1)
]

df = spark.createDataFrame(data, ["feature1", "feature2", "label"])

# 3. Assemble features into a single vector
assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")
df = assembler.transform(df).select("features", "label")

# 4. Train-test split
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)



In [7]:
assembler

VectorAssembler_104c314d60b8

In [8]:
df.show()

+---------+-----+
| features|label|
+---------+-----+
|[1.0,2.0]|    0|
|[2.0,1.0]|    1|
|[1.5,1.5]|    0|
|[3.0,3.0]|    1|
|[2.5,0.5]|    1|
+---------+-----+



In [9]:
train_df.show()

+---------+-----+
| features|label|
+---------+-----+
|[1.0,2.0]|    0|
|[2.0,1.0]|    1|
|[2.5,0.5]|    1|
|[3.0,3.0]|    1|
+---------+-----+



In [10]:
test_df.show()

+---------+-----+
| features|label|
+---------+-----+
|[1.5,1.5]|    0|
+---------+-----+



In [40]:
# 5. Train XGBoost model
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=5)
rf_model = rf.fit(train_df)

25/05/02 08:55:59 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 4 (= number of training instances)


In [41]:
# 6. Inference (Prediction)
predictions = rf_model.transform(test_df)
predictions.show()

+---------+-----+-------------+-----------+----------+
| features|label|rawPrediction|probability|prediction|
+---------+-----+-------------+-----------+----------+
|[1.5,1.5]|    0|  [71.0,29.0]|[0.71,0.29]|       0.0|
+---------+-----+-------------+-----------+----------+



In [15]:
# probability: [0.71,0.29]
# Class 0: 71% confidence
# Class 1: 29% confidence

In [42]:
# Save to a folder (not a file)
rf_model.write().overwrite().save("rf_model")

In [43]:
from pyspark.ml.classification import RandomForestClassificationModel

loaded_rf_model = RandomForestClassificationModel.load("rf_model")

In [44]:
loaded_rf_model.transform(test_df).select("features", "label", "prediction", "probability").show()

+---------+-----+----------+-----------+
| features|label|prediction|probability|
+---------+-----+----------+-----------+
|[1.5,1.5]|    0|       0.0|[0.71,0.29]|
+---------+-----+----------+-----------+

